<a href="https://colab.research.google.com/github/francescodisalvo05/polito-nlp/blob/main/Labs/Lab_01_text_processing_and_topic_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Everything

In [1]:
# install
!pip install fastlangid
!pip install iso639-lang
!pip install langdetect
!pip install langid

In [7]:

import re
import pandas as pd

from iso639 import Lang
from langdetect import detect
from fastlangid.langid import LID
import langid

import spacy
from spacy import displacy

import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

from gensim.corpora import Dictionary
from gensim.models import LdaModel, LsiModel

import pyLDAvis
from pyLDAvis import gensim_models
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# **Deep Natural Language Processing @ PoliTO**

---


**Teaching Assistant:** Moreno La Quatra

**Practice 1:** Text processing and topic modeling

# **Text processing**
---
The text processing phase is a preliminary stage where the text to be manipulated is processed to be ready for subsequent analysis.

Text processing usually entails several steps that could possibly include:
- **Language Identification**: identifying the language of a given text.
- **Tokenization**: splitting a given text in several sentences/words. 
- **Dependency tree parsing:** analyzing the depencies between words composing the text.
- **Stemming/Lemmatization:** obtain the root form for each word in text.
- **Stopword removal**: removing words that are si commonly used that they carry very little useful information.
- **Part of Speech Tagging:** given a word, retrieve its part of speech (proper noun, common noun or verb).



### Language Identification

| Text                                                                                                                                | Language Code |
|-------------------------------------------------------------------------------------------------------------------------------------|---------------|
| The "Deep Natural Language Processing" course is offered during the first semester of the second year at Politecnico di Torino      | `EN`            |
| Il corso "Deep Natural Language Processing" viene impartito al Politecnico di Torino durante il primo semestre del secondo anno.    | `IT`            |
| Le cours "Deep Natural Language Processing" est enseigné au Politecnico di Torino pendant le premier semestre de la deuxième année. | `FR`            |

**Language Identification** is a crucial prelimiary step because each language has its own characteristics. The knowledge of the main language associated to a given text could be beneficial for all subsequent steps in text processing pipeline.

The data collection used in this first part of the practice is provided [here](https://github.com/MorenoLaQuatra/DeepNLP/blob/main/practices/P1/langid_dataset.csv) - [source: Kaggle](https://www.kaggle.com/martinkk5575/language-detection)

# Exercise 1:

Benchmark different language-detection algorithm by computing the accuracy of each approach:
- [FastText](https://pypi.org/project/fastlangid/)
- [LangID](https://github.com/saffsd/langid.py)
- [langdetect](https://pypi.org/project/langdetect/)

**Hint:** language code conversion: [iso639-lang](https://pypi.org/project/iso639-lang/)

For each method report:
- Accuracy
- Average time per example

In [ ]:
!wget https://raw.githubusercontent.com/MorenoLaQuatra/DeepNLP/main/practices/P1/langid_dataset.csv

Your code

In [3]:
df = pd.read_csv('langid_dataset.csv')

y = df.language
X = df.Text

### FastText

In [6]:
%%time

fastlangid = LID()

results_FastText = fastlangid.predict(X)

# there are several dialects for Chineese
results_FastText = ["zh" if x[:2] == "zh" else x for x in results_FastText]
results_name_FastText = [Lang(x).name for x in results_FastText]

print("Accuracy for FastText : ", accuracy_score(y, results_name_FastText))
print("---------------------------------")

Accuracy for FastText :  0.9677272727272728
---------------------------------
CPU times: user 3.58 s, sys: 20.5 ms, total: 3.6 s
Wall time: 3.61 s


### LangID

In [8]:
%%time

results_langid = [Lang(langid.classify(x)[0]).name for x in X]

print("Accuracy for LangID : ", accuracy_score(y, results_langid))
print("--------------------------------------")

Accuracy for LangID :  0.9542727272727273
--------------------------------------
CPU times: user 1min 15s, sys: 53.6 s, total: 2min 9s
Wall time: 1min 7s


### LangDetect

In [9]:
%%time
results_LangDetect = []

# there are four rows that give the error "no features in text"
# one is blank but the other three are Arabic
# >> I will manually omit these ones

errors_idxs = []
y_cleaned = y.copy()

for i,x in enumerate(X):

    try:
        language = detect(x)

        if language.startswith("zh"):
          language = "zh"
        
        results_LangDetect.append(Lang(language).name)

    except:
        # drop this element from y
        y_cleaned.drop(i,inplace=True)

print("Accuracy for LangDetect : ", accuracy_score(y_cleaned, results_LangDetect))
print("--------------------------------------")

Accuracy for LangDetect :  0.8752557166886393
--------------------------------------
CPU times: user 2min 29s, sys: 1.61 s, total: 2min 30s
Wall time: 2min 30s


# Exercise 2

For English-written text, apply word-level tokenization. What is the average number of words per sentence?

Implement word-tokenization using both [nltk](https://www.nltk.org/) and [spacy](https://spacy.io/). Report the results for both of them.

For spaCy use the `en_core_web_sm` model.

### Filter English Text

In [10]:
nlp = spacy.load("en_core_web_sm")

In [11]:
X_eng = df[df["language"] == "English"][["Text"]]

X_eng["num_tokens_nltk"] = df["Text"].apply(lambda x : len(word_tokenize(x)))
X_eng["num_tokens_spacy"] = df["Text"].apply(lambda x : len(nlp(x)))

In [12]:
X_eng.head()

,Text,num_tokens_nltk,num_tokens_spacy
37,in johnson was awarded an american institute ...,28,30
40,bussy-saint-georges has built its identity on ...,40,50
76,minnesotas state parks are spread across the s...,132,144
90,nordahl road is a station served by north coun...,59,59
97,a talk by takis fotopoulos about the internati...,50,53


In [13]:
X_eng.mean(axis=0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


num_tokens_nltk     68.738
num_tokens_spacy    72.334
dtype: float64

# Exercise 3

Use spacy to parse the dependency tree of a **randomly selected** sentence. You can both use English sentences or your native language (if supported in [spaCy](https://spacy.io/usage/models/)). Use [displaCy](https://explosion.ai/demos/displacy) to visualize the result in the notebook.

In [14]:
nlp = spacy.load("en_core_web_sm")

sentence = "Hello, I am Francesco and I am studying at Politecnico!"

doc = nlp(sentence)
displacy.render(doc, style="dep", jupyter=True)

In [15]:
displacy.render(doc, style="ent", jupyter=True)

# Exercise 4
For the same sentence selected in the previous step apply all the following steps:
1. Lemmatization: convert each word to its root form.
2. Stopword removal: remove language-specific stopwords.
3. Part of Speech Tagging: for each word in the sentence display its part-of-speech.

For each step, print the resulting list on the console.

In [ ]:
lemmatizer = WordNetLemmatizer()

lemmatized_tokens = []
cleaned_tokens = []

stopwords_eng = stopwords.words('english')

# change the noun for the lemmatization
sentence = "He is Francesco and he studies at Politecnico"
tokens = word_tokenize(sentence)

# step 0 : print sentence
print(f"Step 0 : {sentence}")

# step 1 : Lemmatization
for token in tokens:
    
    # lemmatization's result is a common sense word
    # therefore we should apply it before the stopword removal

    # lemmatization
    lemmatized_tokens.append(lemmatizer.lemmatize(token))

print(f"Step 1 : {' '.join(lemmatized_tokens)}")


# step 2 : Stopwords removal
for l_token in lemmatized_tokens:    
    
    if l_token not in stopwords_eng:
      cleaned_tokens.append(l_token)

print(f"Step 2 : {' '.join(cleaned_tokens)}")
    
# step 3 : POS tagging
print(f"Step 3 : {nltk.pos_tag(cleaned_tokens)}")

Step 0 : He is Francesco and he studies at Politecnico
Step 1 : He is Francesco and he study at Politecnico
Step 2 : He Francesco study Politecnico
Step 3 : [('He', 'PRP'), ('Francesco', 'NNP'), ('study', 'NN'), ('Politecnico', 'NNP')]


# **Occurrence-based text representation - TF-IDF**

---
TF-IDF (term frequency-inverse document frequency) is a statistical measure that evaluates how relevant a word is to a document in a collection of documents. It allows to create occurrence-based vector representation for each document.

# Exercise 5
Use TF-IDF to vectorize each sentence in the original data collection. You can choose your preferred implementation for TF-IDF vectorization. It is also available on [SciKit-Learn library](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [ ]:
vectorizer = TfidfVectorizer()
X_vec = vectorizer.fit_transform(X)

# Exercise 6

Build a supervised multi-class language detector using as features the vector obtained by TF-IDF representation. Use 80% of the data to train the language detector and 20% of the data for assessing its accuracy.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_vec,y, test_size=0.2, stratify=y)

model = LogisticRegression().fit(X_train,y_train)
results = model.predict(X_test)

print(accuracy_score(y_test,results))

0.9518181818181818


# **Topic Modelling**

Occurrence-based representations are high-dimensional, what is the dimension of the generated TF-IDF vector representation?
Topic modelling focuses on caturing latent topics in large document corpora.

The data collection used in this second part of the practice is provided [here](https://raw.githubusercontent.com/MorenoLaQuatra/DeepNLP/main/practices/P1/CovidFake_filtered.csv) - [source: Zenodo](https://zenodo.org/record/4282522#.YVdCXcbOOpd)


# Exercise 7

Latent Semantic Indexing (LSI) models underlying concepts by using SVD (Singular Value Decomposition).

Use [gensim](https://radimrehurek.com/gensim/) library to:
1. Create a corpus composed of the headlines contained in the data collection.
2. Generate a [dictionary](https://radimrehurek.com/gensim/corpora/dictionary.html) to create a word -> id mapping (required by LSI module).
3. Using the dictionary, preprocess the corpus to obtain the representation required for LSI model training ([documentation here](https://radimrehurek.com/gensim/models/lsimodel.html)).
4. Inspect the top-5 topics generated by the LSI model for the analysed corpus.

In [ ]:
!wget https://raw.githubusercontent.com/MorenoLaQuatra/DeepNLP/main/practices/P1/CovidFake_filtered.csv

In [14]:
def preprocess_text(documents):
  """
  :param df: (Series) documents to clean
  :return: (Series) cleaned documents 
           - Lemmatization
           - Stopwords 
  """

  # tokenizer that removes punctuation
  tokenizer = nltk.RegexpTokenizer(r"\w+")
  lemmatizer = WordNetLemmatizer()
  stopwords_eng = stopwords.words('english')

  cleaned_docs = []

  for curr_doc in documents:

    # lowercase
    curr_doc = curr_doc.lower()
    # tokenize
    curr_tokens = tokenizer.tokenize(curr_doc)
    # lemmatized tokens
    curr_lemmas = [lemmatizer.lemmatize(t) for t in curr_tokens]
    # remove stopwords
    cleaned_tokens = [t for t in curr_lemmas if not t in stopwords_eng]

    cleaned_docs.append(cleaned_tokens)

  return cleaned_docs

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
# step 0
df_covid = pd.read_csv('CovidFake_filtered.csv').drop(columns=['Unnamed: 0'])

# step 1
headlines = df_covid.headlines
headlines_cleaned = preprocess_text(headlines)

# step 2
dct = Dictionary(headlines_cleaned)  # initialize a Dictionary

# step 3

# generate a bag of word corpus
corpus_bow = []
for x in headlines_cleaned:
  corpus_bow.append(dct.doc2bow(x))

# show the first 10 tokens of the first document
print(corpus_bow)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1)], [(0, 1), (2, 1), (5, 1), (9, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)], [(2, 1), (12, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1)], [(0, 1), (5, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1)], [(4, 1), (7, 1), (10, 1), (11, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1)], [(0, 1), (5, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1)], [(4, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1)], [(0, 1), (5, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 2), (88, 1), (89, 1), (90, 2), (91, 1), (92, 1)], [(0, 1), (5, 1), 

In [17]:
# step 4
lsi_model = LsiModel(corpus_bow, id2word=dct, num_topics=5)  
result_lsi = lsi_model.print_topics(num_topics=5, num_words=5)

In [18]:
# example of result
result_lsi[0]

(0,
 '0.608*"covid" + 0.600*"19" + 0.251*"coronavirus" + 0.153*"ha" + 0.138*"claim"')

In [20]:
def clean_print_result(result):
  for res in result:
    
    # extract the most representative words for that topic
    words = re.findall('[a-z]{1,}', res[1])
    joined_words = ', '.join(words)

    print(f"Topic {res[0] + 1} : {joined_words}")

clean_print_result(result_lsi)

Topic 1 : covid, coronavirus, ha, claim
Topic 2 : coronavirus, covid, ha, claim
Topic 3 : coronavirus, claim, post, facebook, ha
Topic 4 : video, show, ha, people, post
Topic 5 : wa, people, video, show, coronavirus


# Exercise 8 (Optional)

The top-scored words contributing to each topic (if no stopword removal is applied) are english common words (e.g., *to, for, in, of, on*..). Repeat the same procedure of Ex. 7 by adding a preliminary preprocessing step to **remove stopwords**.

In [ ]:
# already done above

# Exercise 9 (Optional)

Leveraging the same corpus used for LSI model generation, apply LDA modelling setting the number of topics to 5. Display the words most contributing to the those topics according to the LDA model.

In [21]:
lda_model  = LdaModel(corpus_bow, id2word=dct, num_topics=5)  
result_lda = lda_model.print_topics(num_topics=5, num_words=5)

In [22]:
clean_print_result(result_lda)

Topic 1 : coronavirus, covid, ha, video
Topic 2 : coronavirus, outbreak, show, video, new
Topic 3 : coronavirus, people, covid, lockdown
Topic 4 : coronavirus, china, people, wa, chinese
Topic 5 : covid, coronavirus, cure, ha


# Exercise 10 (Optional)

Using [pyLDAvis]() library build an interactive visualization for the trained LDA model.

In [ ]:
!pip install pyLDAvis==3.3.0

In [24]:
pyLDAvis.enable_notebook()
vis = gensim_models.prepare(lda_model, corpus_bow, dct, mds="mmds", R=15)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.074356  0.096019       1        1  26.575763
4     -0.011284 -0.151235       2        1  24.914481
3     -0.167362 -0.045267       3        1  18.001751
2     -0.073958  0.167242       4        1  16.133402
1      0.178248 -0.066760       5        1  14.374602, topic_info=             Term         Freq        Total Category  logprob  loglift
4     coronavirus  4520.000000  4520.000000  Default  15.0000  15.0000
0              19  2382.000000  2382.000000  Default  14.0000  14.0000
5           covid  2397.000000  2397.000000  Default  13.0000  13.0000
876         china   745.000000   745.000000  Default  12.0000  12.0000
494         water   318.000000   318.000000  Default  11.0000  11.0000
...           ...          ...          ...      ...      ...      ...
78        prevent    80.109960   253.013609   Topic5  -5.2475   0.7897
104            ha   111.169833  1083.613363   Topic5  -4.9198  -0.3373
1129        wuhan    85.715194   392.599339   Topic5  -5.1799   0.4179
5           covid    94.714297  2397.259901   Topic5  -5.0800  -1.2915
0              19    92.219426  2382.914284   Topic5  -5.1067  -1.3122

[220 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         1  0.305508       19
0         2  0.545970       19
0         3  0.036090       19
0         4  0.073439       19
0         5  0.038608       19
...     ...       ...      ...
1129      3  0.583292    wuhan
1129      4  0.028018    wuhan
1129      5  0.219053    wuhan
2166      1  0.939904  youtube
2166      2  0.038363  youtube

[446 rows x 3 columns], R=15, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 4, 3, 2])